In [65]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
from supervised.automl import AutoML

In [66]:
data1 = pd.read_excel('data/FrontiersDS.xlsx', sheet_name='Wide')

In [67]:
data2 = pd.read_excel('data/FrontiersDS.xlsx', sheet_name='Long')

In [68]:
data1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69
0,The data in this sheet was used for the formul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
headings = data1.loc[[2]].values
z = list(headings.flatten())

In [70]:
data1.columns = z

In [71]:
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,The data in this sheet was used for the formul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
data1 = data1.drop([0,1], axis=0)

In [73]:
# forgot to remove a row
data1 = data1.drop([2], axis=0)
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SA,SA_A,SA.A_TSB_C,403286,1850391,NaN,1169501,15,228163,73558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SA,SA_A,SA.A_TSB_D,129833,5140770,NaN,1926072,124282,0,188367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SA,SA_A,SA.A_TSB_E,117105,3422557,NaN,246751,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
data2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [75]:
headings2 = data2.loc[[1]].values
w = list(headings2.flatten())
data2.columns = w
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [76]:
data2 = data2.drop([0], axis=0)
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409


In [77]:
data2 = data2.drop([1], axis=0)
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154


In [78]:
data2

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,n-Tridecan-1-ol,Alcohol,2074±20,55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65,1-Tetradecanol,Alcohol,2165±10 (27),55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,851004,922978,593032,450711,843660,0,78672,84426,146349,635257
66,n-Pentadecanol,Alcohol,2254±15,83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
67,1-Hexadecanol,Alcohol,2382±5 (49),83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,850885,1183943,382201,390122,1033300,0,0,0,0,482026


In [79]:
data2 = data2.rename(columns={np.nan: 'neg'})
data2.head()

,Compounds,Chemical_class,neg,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154


In [80]:
data2.columns

Index([     'Compounds', 'Chemical_class',            'neg',              nan,
           'SA.A_TSB_A',     'SA.A_TSB_B',     'SA.A_TSB_C',     'SA.A_TSB_D',
           'SA.A_TSB_E',     'SA.B_TSB_A',     'SA.B_TSB_B',     'SA.B_TSB_C',
           'SA.B_TSB_D',     'SA.B_TSB_E',     'SA.A_BHI_A',     'SA.A_BHI_B',
           'SA.A_BHI_C',     'SA.A_BHI_D',     'SA.A_BHI_E',     'SA.B_BHI_A',
           'SA.B_BHI_B',     'SA.B_BHI_C',     'SA.B_BHI_D',     'SA.B_BHI_E',
            'SA.A_LB_A',      'SA.A_LB_B',      'SA.A_LB_C',      'SA.A_LB_D',
            'SA.A_LB_E',      'SA.B_LB_A',      'SA.B_LB_B',      'SA.B_LB_C',
            'SA.B_LB_D',      'SA.B_LB_E',     'PA.A_BHI_A',     'PA.A_BHI_B',
           'PA.A_BHI_C',     'PA.A_BHI_D',     'PA.A_BHI_E',     'PA.B_BHI_A',
           'PA.B_BHI_B',     'PA.B_BHI_C',     'PA.B_BHI_D',     'PA.B_BHI_E',
            'PA.A_LB_A',      'PA.A_LB_B',      'PA.A_LB_C',      'PA.A_LB_D',
            'PA.A_LB_E',      'PA.B_LB_A',      'PA.

In [81]:
data2.columns = data2.columns.fillna('SD')
data2

,Compounds,Chemical_class,neg,SD,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,n-Tridecan-1-ol,Alcohol,2074±20,55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65,1-Tetradecanol,Alcohol,2165±10 (27),55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,851004,922978,593032,450711,843660,0,78672,84426,146349,635257
66,n-Pentadecanol,Alcohol,2254±15,83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
67,1-Hexadecanol,Alcohol,2382±5 (49),83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,850885,1183943,382201,390122,1033300,0,0,0,0,482026
